In [ ]:
%pip install requests
%pip install pymssql

In [ ]:
import os
import requests
import pymssql
from datetime import datetime

RDS_SERVER = "database-1.cyf4qyaeu9uh.us-east-1.rds.amazonaws.com"
RDS_USER = "admin"
RDS_PASSWORD = "Makhan50"
RDS_DATABASE = "database-1"

OXR_APP_ID = "407a3d7f81f845298fb8b6c1bf50186c"

CURRENCY_LIST = ["PKR", "EUR", "GBP", "AED", "INR", "AUD", "CAD"]

def fetch_openexchange_data():
    if not OXR_APP_ID:
        raise ValueError("Missing OXR_APP_ID environment variable")
    
    url = f"https://openexchangerates.org/api/latest.json?app_id={OXR_APP_ID}"
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    return resp.json()

def lambda_handler(event=None, context=None):

    # Fetch API
    try:
        data = fetch_openexchange_data()
    except Exception as e:
        return {"status": "error", "message": f"API fetch failed: {str(e)}"}
    
    timestamp = datetime.utcfromtimestamp(data["timestamp"]).strftime("%Y-%m-%d %H:%M:%S")
    base = data["base"]
    rates = data["rates"]

    # Connect to RDS
    try:
        conn = pymssql.connect(
            server=RDS_SERVER,
            user=RDS_USER,
            password=RDS_PASSWORD,
            database=RDS_DATABASE,
            port=1433
        )
        cursor = conn.cursor()
    except Exception as e:
        return {"status": "error", "message": f"RDS connection failed: {str(e)}"}

    # Insert
    try:
        for cur in CURRENCY_LIST:
            if cur in rates:
                cursor.execute("""
                    INSERT INTO openexchange_rates 
                    (timestamp, source, base, currency, rate, status)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (timestamp, "openexchangerates", base, cur, rates[cur], "ok"))
        
        conn.commit()
        cursor.close()
        conn.close()

    except Exception as e:
        return {"status": "error", "message": f"Database insert failed: {str(e)}"}

    return {"status": "success", "rows_inserted": len(CURRENCY_LIST)}


In [3]:
lambda_handler()

{'status': 'error',
 'message': 'Database insert failed: (208, b"Invalid object name \'openexchange_rates\'.DB-Lib error message 20018, severity 16:\\nGeneral SQL Server error: Check messages from the SQL Server\\n")'}